# 下載套件

In [1]:
import torch
import torch.nn as nn
from torch import nn, matmul, softmax
from torch.nn.init import xavier_uniform_
import torch.nn.functional as F
import torch.nn.utils.rnn as rnn_utils
from torch.autograd import Variable

import numpy as np
import pandas as pd
import gensim
from gensim.models import KeyedVectors
import pickle
import gzip 
import gc
import os
import random

In [2]:
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split

from tqdm import tqdm
from collections import Counter

import math

# 參數設置

In [3]:
# 超參數
EPOCHS = 15
LEARNING_RATE = 0.001                  # Learning_rate
BATCH_SIZE = 8
ALPHA = 0.01 
BETA = 0.001 
EMBEDDING_DIMENSION = 32               # 嵌入維度
MODEL_DIMENSION = EMBEDDING_DIMENSION  # 模型維度
HIDDEN_DIMENSION = 128                 # MLP 隱藏層維度
HIDDEN_SIZE = 16                       # LSTM 隱藏層維度
NUM_HEAD = 2
NUM_LAYER = 4

isMC = 1
isI2V = 1

# 匯入檔案
- item2Vec_TaFeng.32d.model
- TaFeng_user_cart_itemid_list.gz
- Ta_feng_clean.csv

In [4]:
# 資料集
# DATASET_NAME = "TaFeng"     # 讀取TaFeng資料
# DATASET_NAME = "Dunnhumby"  # 讀取Dunnhumby資料
DATASET_NAME = "Instacart"  # 讀取Instacart資料

In [5]:
# load word2Vec pre_train model
model_filename = f"../preprocessing-data/item2vec_models/item2vec_{DATASET_NAME}.{EMBEDDING_DIMENSION}d.model"
# {DATASET}
with open(model_filename, "rb") as fp:
    model = pickle.load(fp)
weights = torch.FloatTensor(model.wv.vectors)
weights.shape

torch.Size([27095, 32])

In [6]:
# {DATASET} user_cart_itemid_list 用戶id, 購物籃時間差(不會使用到), 此用戶的購物籃串列(每個串列包含多項目)。
with gzip.open(f"../preprocessing-data/{DATASET_NAME}_user_cart_itemid_list.gz", "rb") as fp:
    user_cart_itemid_list = pickle.load(fp)
user_cart_itemid_list[:2]

[(1,
  [[0, 1, 2, 3, 4],
   [0, 5, 2, 6, 3, 7],
   [0, 2, 5, 8, 9],
   [0, 2, 5, 8, 4],
   [0, 2, 5, 8, 10, 11, 12, 6],
   [0, 2, 5, 8],
   [0, 5, 2, 8, 7],
   [2, 0, 5, 8, 13, 14],
   [14, 13, 8, 0, 5, 2],
   [0, 13, 15, 16, 8, 5, 17, 7, 2],
   [0, 8, 16, 4, 15, 5, 7, 3, 18, 14, 13]],
  [5, 6, 5, 5, 8, 4, 5, 6, 6, 9, 11]),
 (2,
  [[19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 6, 30],
   [31, 32, 33, 20, 34, 35],
   [20, 19, 21, 36, 37],
   [38, 19, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
   [50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 20],
   [19,
    22,
    48,
    39,
    62,
    27,
    44,
    30,
    29,
    45,
    41,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    12,
    34,
    46],
   [19, 22, 25, 62, 34, 39, 45, 44, 70, 71, 72, 30, 12, 40],
   [19, 73, 44, 40, 23, 45, 62, 72, 71, 70, 74, 75, 76, 77, 78, 79],
   [19,
    44,
    22,
    54,
    77,
    36,
    80,
    81,
    82,
    64,
    59,
    60,
    61,
    83,
    66,
    65,
    84,
    24,
  

In [7]:
# Load {DATASET} confidences_Matrix
with gzip.open(f"../preprocessing-data/confidences/{DATASET_NAME}_confidences_array.gz", "rb") as fp:
    confidences = pickle.load(fp)

In [8]:
dataset = pd.read_csv(f"../cleaned_dataset/{DATASET_NAME}_clean.csv")

# 最多購物籃
max_cart_count = dataset.groupby("CUSTOMER_ID")["CART_ID"].nunique().max()
print(max_cart_count)

dataset

100


,CUSTOMER_ID,ORDER_NUMBER,PRODUCT_ID,CART_ID,NEW_ITEM_ID
0,1,1,196,0,0
1,1,1,14084,0,1
2,1,1,12427,0,2
3,1,1,26088,0,3
4,1,1,26405,0,4
...,...,...,...,...,...
3252971,20620,5,22963,295153,74
3252972,20620,5,12204,295153,1604
3252973,20620,5,21543,295153,3295
3252974,20620,5,23541,295153,417


# 切分資料集
- 分成輸入資料與標籤資料
- 訓練集:驗證集:測試集 = 7:1:2

In [9]:
# 切分資料集
# train_set_size = int(len(user_cart_itemid_list) * 0.8)
# valid_set_size = int(len(user_cart_itemid_list) * 0.1)
# test_set_size = len(user_cart_itemid_list)-train_set_size-valid_set_size
# train_set, valid_set, test_set = random_split(user_cart_itemid_list, [train_set_size, valid_set_size, test_set_size])
# print(len(train_set))
# print(len(valid_set))
# print(len(test_set))

In [10]:
# 將切割好的資料集暫存起來

# # 訓練集
# filepath = "../preprocessing-data/TaFeng_dataset/train_set.pkl"
# with open(filepath, "wb") as f:
#     pickle.dump(train_set, f)
# # 驗證集
# filepath = "../preprocessing-data/TaFeng_dataset/valid_set.pkl"
# with open(filepath, "wb") as f:
#     pickle.dump(valid_set, f)
# # 測試集
# filepath = "../preprocessing-data/TaFeng_dataset/test_set.pkl"
# with open(filepath, "wb") as f:
#     pickle.dump(test_set, f)

In [11]:
# 讀取之前暫存的資料集

# 載入訓練、驗證、測試集
with open(f"../preprocessing-data/{DATASET_NAME}_dataset/train_set.pkl", "rb") as fp:
    train_set = pickle.load(fp)
with open(f"../preprocessing-data/{DATASET_NAME}_dataset/valid_set.pkl", "rb") as fp:
    valid_set = pickle.load(fp)
with open(f"../preprocessing-data/{DATASET_NAME}_dataset/test_set.pkl", "rb") as fp:
    test_set = pickle.load(fp)

# Batch

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [13]:
# 定義資料轉換函數(於collate_batch函式中使用)
item_index_pipeline = lambda x : [[model.wv.key_to_index[j] for j in i] for i in x] # 取得購物籃中，項目的索引值(只有使用Item2Vec時會需要)
item_label_pipeline = lambda x : [model.wv.key_to_index[j] for j in x]

In [14]:
class TensorDataset(Dataset):
    # TensorDataset繼承Dataset, 重載__init__, __getitem__, __len__
    # 實現將一組Tensor數據封裝成Tensor數據集
    # 能夠通過Index得到數據集的數據，能夠通過len，得到數據集大小
    def __init__(self, data_tensor):
        self.data_tensor = data_tensor
    def __getitem__(self, index):
        return self.data_tensor[index]
    def __len__(self):
        return len(self.data_tensor)

# 輸出userID, input_list跟label(最後一個購物籃)
def collate_batch(batch):
    # 使用ID、時間差、訓練的購物籃項目、預測的購物籃項目
    userID, input_item_list, label_item_list, input_size_list, label_size_list = [], [], [], [], []
    for _user in batch:
        #　userID
        userID.append(_user[0])
        # 所有購物籃的項目ID串列中的最後一個購物籃項目ID
#         label_item_list.append(torch.tensor(_user[1][-1]))
        label_size_list.append(torch.tensor(_user[2][-1]))
        
        # 不使用Item2Vec進行項目嵌入
        if isI2V == 0:
            train_list = _user[1][0:-1]
            label_list = torch.tensor(_user[1][-1])
        # 使用Item2Vec進行項目嵌入
        else:
            train_list = item_index_pipeline(_user[1][0:-1])
            label_list = torch.tensor(item_label_pipeline(_user[1][-1]))
        input_size_list.append(_user[2][0:-1])
        
        input_item_list.append(train_list) #　所有購物籃的項目ID串列(除了最後一個購物籃)
        label_item_list.append(label_list)
    
    return userID, input_item_list, label_item_list, input_size_list, label_size_list

In [15]:
# 轉成 Dataset
split_train_ = TensorDataset(train_set)
split_valid_ = TensorDataset(valid_set)
split_test_ = TensorDataset(test_set)

In [16]:
# DataLoader 
train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch, drop_last=True)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch, drop_last=True)
test_dataloader = DataLoader(split_test_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch, drop_last=True)

# Self-Attention

In [17]:
# 使用nn.MultiheadAttention
# 輸入一個用戶的一個購物籃，輸出購物籃嵌入
class SelfAttention(nn.Module):
    def __init__(self, embed_dim, model_dim): #項目向量維度，輸出模型的維度
        super(SelfAttention, self).__init__()
        self.embed_dim = embed_dim
        self.model_dim = model_dim
        
        # 初始化Q, K, V 矩陣
        self.query_matrix = nn.Linear(embed_dim, model_dim)
        xavier_uniform_(self.query_matrix.weight)
        self.key_matrix = nn.Linear(embed_dim, model_dim)
        xavier_uniform_(self.key_matrix.weight)
        self.value_matrix = nn.Linear(embed_dim, model_dim)
        xavier_uniform_(self.value_matrix.weight)
        self.multihead_attn = nn.MultiheadAttention(embed_dim, num_heads=1)
        
    def forward(self, inputs, attention_mask):
        
        # 輸入一個項目向量，透過三個可學習的參數矩陣，得到計算所需要的 q,k,v
        q = self.query_matrix(inputs)
        k = self.key_matrix(inputs)
        v = self.value_matrix(inputs)
        
        attn_output, attn_output_weight = self.multihead_attn(q, k, v, key_padding_mask=attention_mask.transpose(0,1))
        
        output_mean = torch.tensor([[0 for _ in range(MODEL_DIMENSION)] for _ in range(len(attention_mask))], dtype=torch.float).to(device)
        for i, cart in enumerate(attention_mask):
            for j, mask in enumerate(cart):
                if mask == False:
                    output_mean[i] = attn_output[i][j] # 使用最後一個項目作為輸出
        basket_embedding = output_mean
        
        return basket_embedding

   # LSTM

In [18]:
class LSTM(nn.Module):
    def __init__(self, input_size, output_size, hiddenlayer1 = 512, hiddenlayer2 = 256, hiddenlayer3 = 128):
        super(LSTM, self).__init__()
        self.lstm = torch.nn.LSTM(input_size, HIDDEN_SIZE, 2)
        self.hiddenlayer1 = torch.nn.Linear(max_cart_count * HIDDEN_SIZE, hiddenlayer1)
        self.hiddenlayer2 = torch.nn.Linear(hiddenlayer1, hiddenlayer1)
        self.hiddenlayer3 = torch.nn.Linear(hiddenlayer1, hiddenlayer2)
        self.hiddenlayer4 = torch.nn.Linear(hiddenlayer2, hiddenlayer3)
        self.embed = torch.nn.Linear(hiddenlayer3, EMBEDDING_DIMENSION)
        self.leakyrelu = torch.nn.LeakyReLU()
    
    def forward(self, inputs):
        output, (h,c) = self.lstm(inputs)
        hidden1 = self.hiddenlayer1(output.view(max_cart_count * HIDDEN_SIZE))
        hidden2 = self.hiddenlayer2(hidden1)
        hidden3 = self.hiddenlayer3(hidden2)
        hidden4 = self.hiddenlayer4(hidden3)
        output = self.embed(hidden4)
        return output

# Transformer

In [19]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model:int, dropout, maxlen:int=500):
        super(PositionalEncoding, self).__init__()
        # den 是把10000^(2i/d_model)取log_e，前面加負號是求倒數
        den = torch.exp(-torch.arange(0, d_model, 2) * math.log(10000) / d_model)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros(maxlen, d_model)#.to(device)
        pos_embedding[:, 0::2] = torch.sin(pos*den)
        pos_embedding[:, 1::2] = torch.cos(pos*den)
        
        pos_embedding = pos_embedding.unsqueeze(0)
        
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("pos_embedding", pos_embedding)
        
    def forward(self, token_embedding):
        return self.dropout(token_embedding+ self.pos_embedding[:, :token_embedding.size(1), :])
    
class TransformerEncoder(nn.Module):
    def __init__(self, d_model, num_heads=8, num_layers=6):
        super(TransformerEncoder, self).__init__()
        self.pe = PositionalEncoding(d_model=d_model, dropout=0.5, maxlen=max_cart_count*2)
        # 創建 Transformer 模型
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=num_heads),
            num_layers=num_layers
        )
    
    def forward(self, baskets_embedding):
        baskets_embedding_pe = self.pe(baskets_embedding)
        
        # 購物籃padding的遮罩
        padding_mask = ~baskets_embedding.sum(dim=-1).ne(0).transpose(0,1)
        
        output = self.transformer(baskets_embedding_pe.to(torch.float32), src_key_padding_mask=padding_mask.to(torch.float32))
        return output

# MLP層

In [20]:
class MLPforItem(nn.Module):
    def __init__(self, embed_dim, hidden_dim, items_dim):
        super(MLPforItem, self).__init__()
        # hidden layer
        self.hidden = nn.Linear(embed_dim, hidden_dim)
        xavier_uniform_(self.hidden.weight)
        self.norm = nn.BatchNorm1d(hidden_dim, momentum=0.03)
        self.relu = nn.ReLU()
        # output layer
        self.output = nn.Linear(hidden_dim, items_dim)
        self.softmax = nn.Softmax(dim=0)
    def forward(self, inputs):
        y = self.relu(self.norm(self.hidden(inputs)))
        return self.output(y)
#         return self.softmax(self.output(y))

In [21]:
class MLPforSize(nn.Module):
    def __init__(self, embed_dim):
        super(MLPforSize, self).__init__()
        # predict layer
        self.predict = nn.Linear(embed_dim, 1)
        self.relu = nn.ReLU()
    def forward(self, inputs):
        output = self.relu(self.predict(inputs))
        return output

# 損失函數

In [22]:
# MSE
def mean_square_error(prediction, target):
    predictions = prediction[0]
    targets = torch.tensor([target[0]], dtype=torch.float).to(device)
    loss = F.mse_loss(predictions, targets)
    for i in range(1, len(prediction)):
        predictions = prediction[i]
        targets= torch.tensor([target[i]], dtype=torch.float).to(device)
        loss += F.mse_loss(predictions, targets)
    return loss

In [23]:
# cross_entropy_loss
def cross_entropy_loss(predictions, targets):
    # 創建稀疏張量的索引和值
    indices = []
    values = []
    for i, t in enumerate(targets):
        for v in t:
            indices.append([i, v])
            values.append(1)
            
    # 創建稀疏張量
    sparse_targets = torch.sparse_coo_tensor(indices=torch.tensor(indices).t(),
                                             values=torch.tensor(values, dtype=torch.float32),
                                             size=(len(targets), items_count), device=device)
    sparse_targets = sparse_targets.to_dense()
    
    loss = F.binary_cross_entropy_with_logits(predictions, sparse_targets)
    return loss

# 評估指標

In [24]:
def format_metric(result_dict):
    assert type(result_dict) == dict
    format_str = []
    metrics = np.unique([k for k in result_dict.keys()])
    for metric in metrics:
        name = '{}'.format(metric)
        m = result_dict[name]
        if type(m) is float or type(m) is np.float32 or type(m) is np.float64:
            format_str.append("{}: {:<.4f}".format(name, m))
        elif type(m) is int or type(m) is np.int32 or type(m) is np.int64:
            format_str.append("{}: {}".format(name, m))
    return ", ".join(format_str)

## F1-score

In [25]:
def calculate_f1_score(predictions, targets, k_list):
    """
    計算 F1-score@K。

    Args:
        predictions: 二維的預測機率矩陣，大小為 [num_users, num_items]。
        targets: 一個包含每個用戶真實標籤的列表，其中每個列表的大小不同。
        k_list: 用預測出的K值，計算F1-score@K

    Returns:
        F1-score@K 分數。
    """
    # 將預測機率矩陣值轉換為 Pytorch 張量
    predictions = torch.from_numpy(np.array(predictions, dtype=np.float32))
    num_users = len(targets)
    f1_score_eval = dict()
    
    precision_sum = 0.0
    recall_sum = 0.0
    f1_score_sum = 0.0
    for i in range(num_users):
        
        # 將用戶 i 的真實標籤轉換為 PyTorch 張量。
        labels = torch.from_numpy(np.array(targets[i], dtype=np.int64))
        # 計算用戶 i 在預測機率矩陣中機率最高的 K 個項目索引
        top_k_item_labels = torch.topk(predictions[i], k_list[i])[1]
        # 計算用戶 i 的真實標籤和預測標籤的交集 (TP)
        true_positives = torch.sum(torch.sum(torch.eq(top_k_item_labels, labels.unsqueeze(1)).to(torch.float32), dim=1)).item()
        # 計算用戶 i 的真實標籤和預測標籤的聯集
        predicted_positives = k_list[i] # TP+FP
        actual_positives = len(labels)  # TP+FN
        # 預防 K 預測為0導致 precision 分母為0
        if predicted_positives == 0:
            precision = 0.0
        else:
            precision = true_positives / predicted_positives
        # 預防實際 K 為0導致 recall 分母為0
        if actual_positives == 0:
            recall = 0.0
        else:
            recall = true_positives / actual_positives
        # 計算F1-score
        if precision + recall == 0:
            f1_score = 0.0
        else:
            f1_score = 2 * precision * recall / (precision + recall)
        precision_sum += precision
        recall_sum += recall
        f1_score_sum += f1_score
        
    # 計算平均分數
    precision = precision_sum / float(num_users)
    recall = recall_sum / float(num_users)
    f1_score = f1_score_sum / float(num_users)
    # 儲存到dict
    key = "{}".format("Recall")
    f1_score_eval[key] = recall
    key = "{}".format("Precision")
    f1_score_eval[key] = precision
    key = "{}".format("F1-score")
    f1_score_eval[key] = f1_score
    
    return f1_score_eval

## NDCG

In [26]:
def calculate_ndcg(basket_predictions, basket_targets, size_predictions, size_targets):
    """
    計算 NDCG@K。

    Args:
        basket_predictions: 預測購物籃項目
        bakset_targets: 實際購物籃項目
        size_predictions: 預測購物籃大小
        size_targets: 實際購物籃大小  

    Returns:
        NDCG@K 分數。
    """
    # 將預測機率矩陣轉換為 PyTorch 張量
    predictions = torch.from_numpy(np.array(basket_predictions, dtype=np.float32))
    num_users = len(basket_targets)
    ndcg_eval = dict()
    
    ndcg_sum = 0.0
    for i in range(num_users):
        # 將用戶 i 的真實標籤轉換為 PyTorch 張量
        labels = torch.from_numpy(np.array(basket_targets[i], dtype=np.int64))
        # 計算用戶 i 在預測機率矩陣中機率最高的 K 個項目的索引=標籤
        top_k_item_labels = torch.topk(basket_predictions[i], size_predictions[i])[1]
        # 計算 DCG@K
        dcg_at_k = torch.sum(torch.nan_to_num(torch.div(1.0, torch.log2(torch.arange(size_predictions[i], dtype=torch.float32) +2))) * (torch.eq(top_k_item_labels, labels.unsqueeze(1)).to(torch.float32)))
        # 計算 IDCG@K
        idcg_at_k = torch.sum(torch.div(1.0, torch.log2(torch.arange(len(labels), dtype=torch.float32) + 2)))
        # 計算 NDCG@K * Penalty weight
        if torch.eq(idcg_at_k, 0):
            ndcg_at_k = idcg_at_k
        else:
            ndcg_at_k = (dcg_at_k / idcg_at_k) * (size_targets[i] / (size_targets[i] + abs(size_targets[i] - size_predictions[i])))
        ndcg_sum += ndcg_at_k.item()
    #　計算平均　NDCG@K 分數
    ndcg = ndcg_sum / float(num_users)
    key = "{}".format("NDCG")
    ndcg_eval[key] = ndcg

    return ndcg_eval

In [27]:
def calculate_mae(size_predictions, size_targets):
    sum = 0
    num_users = len(size_targets)
    mae_eval = dict()
    for i in range(num_users):
        sum += abs(size_predictions[i] - (size_targets[i]).item())
    key = "{}".format("MAE")
    mae_eval[key] = sum / num_users
    return mae_eval

# 訓練&測試

In [28]:
# 訓練模型
def train_model():
    my_model.train()
    loss_list = []
    
    for batch_idx, (userID, basket_input, basket_label, size_input, size_label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        basket_output, size_output = my_model(basket_input, size_input)
        # 計算損失
        loss = ALPHA * mean_square_error(size_output, size_label) + (1 - ALPHA) * cross_entropy_loss(basket_output, basket_label)
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
        
        if (batch_idx%100 == 0) or (batch_idx == len(train_dataloader)-1):
            precentage = (100 * batch_idx/len(train_dataloader))
            print(f"Epoch {epoch}: {precentage:.0f}%, loss: {loss.item():.6f}")
            
        with torch.no_grad():
            basket_output = torch.from_numpy(np.array(basket_output.cpu(), dtype=np.float32))
            size_output = np.round(np.squeeze(np.array([_.cpu() for _ in size_output], dtype=np.float32))).astype(int).tolist()
            if batch_idx == 0:
                basket_outputs = basket_output
                basket_labels = basket_label
                size_outputs = size_output
                size_labels = size_label
            else:
                basket_outputs = torch.cat( (basket_outputs, basket_output),-2 )
                basket_labels = basket_labels + basket_label
                size_outputs = size_outputs + size_output
                size_labels = size_labels + size_label
                
    with torch.no_grad():
        evaluations = calculate_f1_score(basket_outputs, basket_labels, size_outputs) 
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")

        evaluations = calculate_ndcg(basket_outputs, basket_labels, size_outputs, size_labels) 
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")

        evaluations = calculate_mae(size_outputs, size_labels)
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")
        
    return torch.mean(torch.tensor(loss_list))

In [29]:
# 驗證模型
def evaluate_model():
    my_model.eval()
    loss_list = []
    for batch_idx, (userID, basket_input, basket_label, size_input, size_label) in enumerate(tqdm(valid_dataloader)):
        basket_output, size_output= my_model(basket_input, size_input)
        # 計算損失
        loss = ALPHA * mean_square_error(size_output, size_label) + (1 - ALPHA) * cross_entropy_loss(basket_output, basket_label)
        loss_list.append(loss.item())
        
        with torch.no_grad():
            basket_output = torch.from_numpy(np.array(basket_output.cpu(), dtype=np.float32))
            size_output = np.round(np.squeeze(np.array([_.cpu() for _ in size_output], dtype=np.float32))).astype(int).tolist()
            if batch_idx == 0:
                basket_outputs = basket_output
                basket_labels = basket_label
                size_outputs = size_output
                size_labels = size_label
            else:
                basket_outputs = torch.cat( (basket_outputs, basket_output),-2 )
                basket_labels = basket_labels + basket_label
                size_outputs = size_outputs + size_output
                size_labels = size_labels + size_label
        
    with torch.no_grad():
        evaluations = calculate_f1_score(basket_outputs, basket_labels, size_outputs) 
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")

        evaluations = calculate_ndcg(basket_outputs, basket_labels, size_outputs, size_labels) 
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")

        evaluations = calculate_mae(size_outputs, size_labels)
        res_str = '(' + format_metric(evaluations) + ')'
        print(f"                      {res_str}\n")
        
    return torch.mean(torch.tensor(loss_list))

In [30]:
# 測試模型
def test_model():
    my_model.eval()
    loss_list = []
    for batch_idx, (userID, basket_input, basket_label, size_input, size_label) in enumerate(tqdm(test_dataloader)):
        basket_output, size_output = my_model(basket_input, size_input)
        # 計算損失
        loss = ALPHA * mean_square_error(size_output, size_label) + (1 - ALPHA) * cross_entropy_loss(basket_output, basket_label)
        loss_list.append(loss.item())
        
        with torch.no_grad():
            basket_output = torch.from_numpy(np.array(basket_output.cpu(), dtype=np.float32))
            size_output = np.round(np.squeeze(np.array([_.cpu() for _ in size_output], dtype=np.float32))).astype(int).tolist()
            if batch_idx == 0:
                basket_outputs = basket_output
                basket_labels = basket_label
                size_outputs = size_output
                size_labels = size_label
            else:
                basket_outputs = torch.cat( (basket_outputs, basket_output),-2 )
                basket_labels = basket_labels + basket_label
                size_outputs = size_outputs + size_output
                size_labels = size_labels + size_label
    
    with torch.no_grad():
        f1_evaluations = calculate_f1_score(basket_outputs, basket_labels, size_outputs)
        precision_list = [f1_evaluations["Precision"]]
        recall_list =  [f1_evaluations["Recall"]]
        f1_list = [f1_evaluations["F1-score"]]
        res_str = '(' + format_metric(f1_evaluations) + ')'
        print(f"                      {res_str}\n")
        
        ndcg_evaluations = calculate_ndcg(basket_outputs, basket_labels, size_outputs, size_labels) 
        ndcg_list = [ndcg_evaluations["NDCG"]]
        res_str = '(' + format_metric(ndcg_evaluations) + ')'
        print(f"                      {res_str}\n")
        
        mae_evaluations = calculate_mae(size_outputs, size_labels)
        mae_list = [mae_evaluations["MAE"]]
        res_str = '(' + format_metric(mae_evaluations) + ')'
        print(f"                      {res_str}\n")
        
    return torch.mean(torch.tensor(loss_list)), recall_list, precision_list, f1_list, ndcg_list, mae_list

# MC

In [31]:
def create_transition_matrix(baskets, items):
    # 統計所有項目在前一個購物籃和下一個購物籃中出現的次數，
    # 以及在前一個購物籃中出現的總次數
    cooccur_counts = {}
    prev_basket_counts = {}
    for user_baskets in baskets:
        for i in range(1, len(user_baskets)):
            prev_basket = user_baskets[i-1]
            cur_basket = user_baskets[i]
            for item1 in prev_basket:
                if item1 not in prev_basket_counts:
                    prev_basket_counts[item1] = 0
                prev_basket_counts[item1] += 1
                for item2 in cur_basket:
                    if item2 not in cooccur_counts:
                        cooccur_counts[item2] = {}
                    if item1 not in cooccur_counts[item2]:
                        cooccur_counts[item2][item1] = 0
                    cooccur_counts[item2][item1] += 1

    # 將統計數據轉換為轉移矩陣
    # items = sorted(list(prev_basket_counts.keys()))
    num_items = len(items)
    transition_matrix = np.zeros((num_items, num_items))
    for i, item1 in tqdm(enumerate(items)):
        for j, item2 in enumerate(items):
            if item2 in cooccur_counts and item1 in cooccur_counts[item2]:
                cooccur_count = cooccur_counts[item2][item1]
                prev_basket_count = prev_basket_counts[item1]
                transition_matrix[i, j] = cooccur_count / prev_basket_count
    return transition_matrix

In [32]:
def predict_next_basket(users_baskets, transition_matrix, items, top_n=50):
    num_users = len(users_baskets)
    num_items = len(items)
    predictions = []
    pred_label = []
    for i in range(num_users):
        last_basket = users_baskets[i][-1] # 第i個用戶的最後一個購物籃
        cur_prediction = np.zeros(num_items)
        prev_item_idx = np.array([prev_item for prev_item in last_basket])
        for j in range(num_items):
            prob = np.sum(transition_matrix[prev_item_idx, j])
            cur_prediction[j] = prob * (1/len(last_basket))
        predictions.append(cur_prediction)
    return predictions

In [33]:
mc_item = dataset["NEW_ITEM_ID"].unique() # 所有項目

In [34]:
train_list = []
for batch_idx, (userID, basket_input, basket_label, size_input, size_label) in enumerate(train_dataloader):
    for i in basket_input:
        train_list.append(i)

In [35]:
# 建立所有用戶共用的轉移矩陣
transition_matrix= create_transition_matrix(train_list, mc_item)
transition_array = torch.tensor(transition_matrix, dtype=torch.float64).to(device)
print(f"transition_matrix={transition_array}")

27095it [04:21, 103.72it/s]


transition_matrix=tensor([[0.5876, 0.0289, 0.1019,  ..., 0.0000, 0.0000, 0.0000],
        [0.0358, 0.5531, 0.1494,  ..., 0.0000, 0.0000, 0.0000],
        [0.1761, 0.2094, 0.4203,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)


# 完整模型

In [36]:
# 項目總數
items_count = confidences.shape[0]
print("items_count=", items_count)
# 項目出現次數
items_freq = Counter(dataset["NEW_ITEM_ID"])
# 計算每個項目出現的比例: items_frq/items_count
item_weight = torch.tensor(np.array(list(items_freq.values()))/items_count).to(device)
# 按照new_item_id順序排列
print(item_weight)

items_count= 27095
tensor([0.1097, 0.0556, 0.0193,  ..., 0.0001, 0.0002, 0.0002], device='cuda:0',
       dtype=torch.float64)


## 加上信賴度矩陣

In [37]:
# 信賴度矩陣
confidences_array = torch.tensor(confidences, dtype=torch.float64).to(device)
confidences_array

tensor([[0.0000, 0.0022, 0.1153,  ..., 0.0000, 0.0000, 0.0000],
        [0.0028, 0.0000, 0.1653,  ..., 0.0000, 0.0000, 0.0000],
        [0.2023, 0.2322, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0', dtype=torch.float64)

In [38]:
def Interleave(tensor1, tensor2):
    tensor2 = tensor2.repeat(tensor1.shape[0], 1)
    result = torch.stack((tensor1, tensor2), dim=1)
    Interleave_tensor = torch.reshape(result, (-1, MODEL_DIMENSION))

    return (Interleave_tensor)

In [39]:
class MyModel01(nn.Module):
    def __init__(self, embed_dim, model_dim, hidden_dim, items_count):
        super(MyModel01, self).__init__()
        self.model_dim = model_dim
        self.embedding = nn.Embedding.from_pretrained(weights, freeze=False)
        self.embedding.requires_grad=True
        self.basket_embed = SelfAttention(embed_dim=embed_dim, model_dim=model_dim)
        self.size_embed = LSTM(1,1)
        self.model_encoder = TransformerEncoder(d_model=model_dim, num_heads=NUM_HEAD, num_layers=NUM_LAYER)
        # 嵌入維度、隱藏層維度、總項目數量
        self.basket_mlp = MLPforItem(model_dim, hidden_dim, items_count)
        self.size_mlp = MLPforSize(model_dim)
        self.relu = nn.ReLU()
        
    def forward(self, basket_input, size_input):
        
        basket_list, size_list, attention_mask, k_list = [], [], [], []
        output_list = []
        
        # 為每個用戶的購物籃加上 padding跟 mask
        for user in basket_input:
            # 將購物籃項目 ID 轉換為嵌入向量
            batch_features = [ self.embedding(torch.tensor(cart).to(device)) for cart in user ]
            # 進行 padding
            batch_features = rnn_utils.pad_sequence(batch_features, batch_first=True, padding_value=0)
            # 購物籃中項目的遮罩
            mask = ~batch_features.sum(dim=-1).ne(0)
            basket_list.append(batch_features)
            attention_mask.append(mask)
        
        # 預測size_tensor
        sizes_input = [torch.tensor(size).to(device) for size in size_input]
        tmp_tensor = torch.zeros(max_cart_count)
        size_list.append(tmp_tensor)
        for size in sizes_input:
            size_list.append(size)
        size_list = rnn_utils.pad_sequence(size_list, batch_first=True, padding_value=0)[1:]
            
        # 進入自注意力，輸出形狀(BATCH_SIZE, basket_size, embed_dim)
        basket_embedding_list = []
        for i,user_inputs in enumerate(basket_list):
            basketEmbed = self.basket_embed(user_inputs, attention_mask[i])
            sizeEmbed = self.size_embed(torch.tensor([[float(_)] for _ in size_list[i]]).to(device))
            
            basket_embedding = torch.cat((basketEmbed, sizeEmbed.reshape(1,-1)), 0)
            
            k_list.append(sizeEmbed)
            basket_embedding_list.append(basket_embedding)
            
        
        # 進行購物籃的 padding
        input_seq = rnn_utils.pad_sequence(basket_embedding_list, batch_first=True, padding_value=0)
        #print(input_seq.size())
        
        
        # 進入Transformer
        basket_embed = self.model_encoder(input_seq.to(device))
        
        B_s_list = []
        K_s_list = []
        for i, b in enumerate(basket_embed):
            basket_size = len(attention_mask[i])
            B_s = b[basket_size-1]  # 取得最後一個購物籃向量
            B_s_list.append(B_s)
            K_s_list.append(B_s+k_list[i])
            
        # 進入basket MLP層
        p = self.basket_mlp(torch.stack(B_s_list, dim=0))
        predictions = predict_next_basket(basket_input, transition_matrix, list(mc_item), 10)
        mc_predictions = torch.tensor(np.array(predictions), dtype=torch.float64).to(device)
        pc = (self.relu(p.to(torch.float64))+1e-8) @ confidences_array
        pw = torch.mul( p, item_weight )
        
        # z-score for mc_predictions
        mc_mean = mc_predictions.mean(dim=1, keepdim = True)
        mc_std = mc_predictions.std(dim=1, keepdim =True)
        mc_z_score = (mc_predictions - mc_mean) / mc_std
        
        # z-score for mc_predictions
        tr_predictions = torch.add(pc, pw) + p.to(torch.float64)
        tr_mean = tr_predictions.mean(dim=1, keepdim = True)
        tr_std = tr_predictions.std(dim=1, keepdim =True)
        tr_z_score = (tr_predictions - tr_mean) / tr_std
        
        p_ = torch.mul(BETA, tr_predictions) + torch.mul((1-BETA), mc_z_score + tr_z_score)
        
        # 進入size MLP層
        k = self.size_mlp(torch.stack(K_s_list, dim=0))
        
        return p_, k

In [40]:
my_model = MyModel01(embed_dim=EMBEDDING_DIMENSION, model_dim=MODEL_DIMENSION, hidden_dim=HIDDEN_DIMENSION, items_count=items_count).to(device)
optimizer = torch.optim.Adam(my_model.parameters(), lr=LEARNING_RATE)
my_model.train()

MyModel01(
  (embedding): Embedding(27095, 32)
  (basket_embed): SelfAttention(
    (query_matrix): Linear(in_features=32, out_features=32, bias=True)
    (key_matrix): Linear(in_features=32, out_features=32, bias=True)
    (value_matrix): Linear(in_features=32, out_features=32, bias=True)
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=32, out_features=32, bias=True)
    )
  )
  (size_embed): LSTM(
    (lstm): LSTM(1, 16, num_layers=2)
    (hiddenlayer1): Linear(in_features=1600, out_features=512, bias=True)
    (hiddenlayer2): Linear(in_features=512, out_features=512, bias=True)
    (hiddenlayer3): Linear(in_features=512, out_features=256, bias=True)
    (hiddenlayer4): Linear(in_features=256, out_features=128, bias=True)
    (embed): Linear(in_features=128, out_features=32, bias=True)
    (leakyrelu): LeakyReLU(negative_slope=0.01)
  )
  (model_encoder): TransformerEncoder(
    (pe): PositionalEncoding(
      (dropout): Dropout

In [41]:
results = []

for epoch in range(1, EPOCHS + 1):
    train_loss = train_model()
    print("train_loss=", train_loss)
    print("-"*20)
    val_loss = evaluate_model()
    print("val_loss=", val_loss)
    print("-"*20)
    test_loss, recall_list, precision_list, f1_list, ndcg_list, mae_list = test_model()
    print("-"*20)
    result = [epoch] + recall_list + precision_list + f1_list + ndcg_list + mae_list + [test_loss.item()]
    results.append(result)
    print(results)
    print("-"*89)
    
    collected = gc.collect()
    torch.cuda.empty_cache()
    
record_df = pd.DataFrame(results, columns=["Epoch", "Recall", "Precision", "F1-score", "NDCG", "MAE", "Loss"])

result_folder = "../result"
if not os.path.exists(result_folder):
    os.mkdir(result_folder)
record_df.to_csv(result_folder+f"/{DATASET_NAME}_output.csv", index=False)

record_df


  0%|          | 1/1948 [00:01<47:54,  1.48s/it]

Epoch 1: 0%, loss: 18.055773


  5%|▌         | 101/1948 [01:40<31:58,  1.04s/it]

Epoch 1: 5%, loss: 6.579185


 10%|█         | 201/1948 [03:23<28:52,  1.01it/s]

...

100%|██████████| 243/243 [03:09<00:00,  1.28it/s]


                      (F1-score: 0.1906, Precision: 0.1858, Recall: 0.2204)

                      (NDCG: 0.1693)

                      (MAE: 4.4177)

val_loss= tensor(2.8008)
--------------------


100%|██████████| 243/243 [03:10<00:00,  1.28it/s]


                      (F1-score: 0.1847, Precision: 0.1817, Recall: 0.2106)

                      (NDCG: 0.1671)

                      (MAE: 4.3282)

--------------------
[[1, 0.1875991175455632, 0.19108291666747096, 0.18043854934274806, 0.1457448609637043, 4.029320987654321, 3.6180145740509033], [2, 0.20539651857809835, 0.17557436896824768, 0.1800687695120412, 0.1471341559835882, 4.262345679012346, 3.475299119949341], [3, 0.18758900651202198, 0.16941856153482818, 0.16979553941254444, 0.14120907373889255, 4.294238683127572, 3.451737403869629], [4, 0.1909076314574276, 0.1858205367875703, 0.17968947492653034, 0.1503059285517529, 3.9902263374485596, 2.970322847366333], [5, 0.20918898828584626, 0.1765620068271424, 0.1822218291935278, 0.16676721420453347, 4.290637860082304, 2.8762545585632324], [6, 0.2094318462531283, 0.18161840783927125, 0.18569726459028094, 0.17061337337200624, 4.161522633744856, 2.780850410461426], [7, 0.21033745664572803, 0.18550087494313844, 0.18818766645239998, 0.17

,Epoch,Recall,Precision,F1-score,NDCG,MAE,Loss
0,1,0.187599,0.191083,0.180439,0.145745,4.029321,3.618015
1,2,0.205397,0.175574,0.180069,0.147134,4.262346,3.475299
2,3,0.187589,0.169419,0.169796,0.141209,4.294239,3.451737
3,4,0.190908,0.185821,0.179689,0.150306,3.990226,2.970323
4,5,0.209189,0.176562,0.182222,0.166767,4.290638,2.876255
5,6,0.209432,0.181618,0.185697,0.170613,4.161523,2.780850
6,7,0.210337,0.185501,0.188188,0.173157,4.113683,2.744442
7,8,0.207005,0.188738,0.187768,0.171433,4.119342,2.825496
8,9,0.206453,0.192017,0.190298,0.175166,3.977881,2.683501
9,10,0.201957,0.199141,0.191172,0.175756,4.019547,2.713016
